In [ ]:
import requests
import pandas as pd
import mysql.connector


In [ ]:
# Extract top 10 anime data
response = requests.get("https://api.jikan.moe/v4/top/anime")
data = response.json()

# Take just the 'data' part of the JSON
anime_list = data['data']

# Convert to DataFrame
df = pd.json_normalize(anime_list)
df.head()


In [ ]:
# Select useful columns
df_clean = df[['title', 'type', 'episodes', 'score', 'rank', 'popularity', 'members']]
df_clean.dropna(inplace=True)
df_clean.head()


In [ ]:
df_clean.to_csv('Anime_ETL_cleaned.csv', index=False)


In [ ]:
# Replace with your MySQL credentials
connection = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password",
    database="anime_db"
)

cursor = connection.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS anime_top (
    title VARCHAR(255),
    type VARCHAR(50),
    episodes INT,
    score FLOAT,
    rank INT,
    popularity INT,
    members INT
)
""")

# Insert data
for index, row in df_clean.iterrows():
    cursor.execute("""
    INSERT INTO anime_top (title, type, episodes, score, rank, popularity, members)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

connection.commit()
connection.close()
